In [1]:
import torch
import torch.nn.functional as F
from torch import nn
from torch.nn import Sequential, Linear, ReLU, CrossEntropyLoss
from torch.optim import Adam
from torch.utils.data.dataset import random_split

import GCL.augmentors as A
import GCL.losses as L
from GCL.models import DualBranchContrast

from torch_geometric.nn import GINConv, global_add_pool
from torch_geometric.loader import DataLoader
from torch_geometric.datasets import TUDataset
from torch_geometric.utils import dropout_edge, add_random_edge, to_dense_adj

from wgin_conv import WGINConv

from tqdm import tqdm
import itertools
import warnings
import sys
from sklearn.model_selection import StratifiedKFold
import numpy as np
import os.path as osp

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
dataset_name = 'REDDIT-BINARY'
device = torch.device('cuda')
path = osp.join(osp.expanduser('~'), 'datasets')
dataset = TUDataset(path, name=dataset_name)
one_graph = dataset[0]

adj, features, labels = (to_dense_adj(one_graph.edge_index).squeeze().to(device), one_graph.x.to(device), one_graph.y.to(device))
print(adj.device)
print(features.device)
print(labels.device)

AttributeError: 'NoneType' object has no attribute 'to'

In [9]:
a = [0.7650, 0.7350, 0.7700, 0.7400]
a = torch.tensor(a)
print(a.mean(), a.std())

tensor(0.7525) tensor(0.0176)


In [66]:
# Prepare the victim model
from gin import *

dataset_name = 'REDDIT-BINARY'
train_multiple_classifiers = False

# Hyperparams
lr = 0.01
num_layers = 3
epochs = 20
print(f'======The hyperparams: lr={lr}, num_layers={num_layers}, epochs={epochs}. On dataset:{dataset_name}======')

device = torch.device('cuda')
path = osp.join(osp.expanduser('~'), 'datasets')
dataset = TUDataset(path, name=dataset_name)
dataloader = DataLoader(dataset, batch_size=128, shuffle=True)
num_features = max(dataset.num_features, 1)
num_classes = dataset.num_classes
if dataset.num_features==0 :
    print("No node feature, paddings of 1 will be used in GIN when forwarding.")


best_hyperparams = {'lr': 0.01, 'num_layer': 5, 'hidden_dim': 32, 'dropout': 0.5, 'batch_size': 128}
lr, num_layer, hidden_dim, dropout, batch_size = best_hyperparams.values()

train_val_set, eval_set = random_split(dataset, [0.9, 0.1])
dataloader_train = DataLoader(train_val_set, batch_size=batch_size, shuffle=True) # Use all train+val to train the final model
dataloader_eval = DataLoader(eval_set, batch_size=128, shuffle=False) # Do not shuffle the evaluation set to make it reproduceable



encoder_model = GIN(num_features=num_features, dim=hidden_dim, num_gc_layers=num_layer, dropout=dropout).to(device)
classifier = LogReg(hidden_dim * num_layer, num_classes).to(device)
model = GCL_classifier(encoder_model, classifier)
optimizer = Adam(model.parameters(), lr=lr)
scheduler = StepLR(optimizer, step_size=50, gamma=0.5)

# Train the encoder with full dataset without labels using contrastive learning
with tqdm(total=epochs, desc='(T)') as pbar:
    for epoch in range(1, epochs + 1):
        loss = train(model, dataloader_train, optimizer, scheduler)
        pbar.set_postfix({'loss': loss})
        pbar.update()

# Accuracy on the clean evaluation data
acc_clean, mask = eval_encoder(model, dataloader_eval)

# Instantiate an attacker and attack the victim model
attacker = Greedy(pn=0.05)
dataloader_eval_adv = attacker.attack(eval_set, mask)

# Accuracy on the adversarial data only
acc_adv_only, _ = eval_encoder(model, dataloader_eval_adv)

# Overall adversarial accuracy
acc_adv = acc_clean * acc_adv_only # T/all * Tadv/T = Tadv/all

print(f'(A): clean accuracy={acc_clean:.4f}, adversarial accuracy={acc_adv:.4f}')


======The hyperparams: lr=0.01, num_layers=3, epochs=20. On dataset:REDDIT-BINARY======
No node feature, paddings of 1 will be used in GIN when forwarding.


(T): 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 20/20 [00:05<00:00,  3.56it/s, loss=42.1]


NameError: name 'device' is not defined

In [5]:
import torch_geometric
from attacker.PGD import PGDAttack

In [6]:
attacker = PGDAttack(surrogate=model, device=device)

In [7]:
one_graph.to(device)

Data(edge_index=[2, 162], x=[42, 3], y=[1])

In [8]:
model(one_graph.x, one_graph.edge_index, one_graph.batch)

tensor([[-0.9111, -3.5493]], device='cuda:0', grad_fn=<AddmmBackward0>)

In [9]:
fc_edge_index, edge_weight = attacker.preprocess(one_graph.x, one_graph.edge_index)

In [10]:
fc_edge_index

tensor([[ 0,  0,  0,  ..., 41, 41, 41],
        [ 1,  2,  3,  ..., 38, 39, 40]], device='cuda:0')

In [11]:
edge_weight

tensor([0., 0., 0.,  ..., 0., 0., 1.], device='cuda:0')

In [12]:
model(one_graph.x, fc_edge_index, one_graph.batch, edge_weight=edge_weight)

tensor([[-0.9111, -3.5493]], device='cuda:0', grad_fn=<AddmmBackward0>)

In [13]:
logit = model(one_graph.x, fc_edge_index, one_graph.batch, edge_weight=edge_weight)
loss = F.cross_entropy(logit, one_graph.y)
print(loss.item())

0.06904802471399307


### 上面已经成功地进行了改造，接下来求取对于edge_weight的梯度

In [14]:
n_perturbations = int(one_graph.x.shape[0] * 0.05)
fc_edge_index, modified_weight, adv_edge_index = attacker.attack_one_graph(one_graph.x, one_graph.edge_index, one_graph.batch, one_graph.y, n_perturbations)

In [15]:
adv_edge_index.shape

torch.Size([2, 160])

In [16]:
logit = model(one_graph.x, adv_edge_index, one_graph.batch)
loss = F.cross_entropy(logit, one_graph.y)
print(loss.item())

0.14834341406822205


In [17]:
logit = model(one_graph.x, fc_edge_index, one_graph.batch, edge_weight=modified_weight)
loss = F.cross_entropy(logit, one_graph.y)
print(loss.item())

0.14834341406822205


In [18]:
print(fc_edge_index.shape)
print(adv_edge_index.shape)

torch.Size([2, 1722])
torch.Size([2, 160])


### 已经成功实现了PGD，接下来将整个test dataset进行转换

In [54]:
from torch_geometric.data import Data

In [55]:
attacker = PGDAttack(surrogate=model, device=device)
attack_ratio = 0.05

In [56]:
dataset_eval = dataset[eval_set.indices]

In [57]:
onegraph = dataset_eval[0]
onegraph

Data(edge_index=[2, 286], x=[77, 3], y=[1])

In [52]:
adv_datalist_eval = []

for one_graph in tqdm(dataset_eval):
    one_graph.to(device)
#     print(f'before the attack: {one_graph}')
    n_perturbations = int(one_graph.x.shape[0] * attack_ratio)
    attacker = PGDAttack(surrogate=model, device=device)
    _, _, adv_edge_index = attacker.attack_one_graph(one_graph.x, one_graph.edge_index, one_graph.batch, one_graph.y, n_perturbations)
    new_graph = Data(edge_index=adv_edge_index, x=one_graph.x, y=one_graph.y)
#     print(f'after the attack: {new_graph}')
    adv_datalist_eval.append(new_graph)

assert len(adv_datalist_eval)==len(dataset_eval),"haha"

  1%|█▍                                                                                                                                                         | 1/111 [00:02<04:51,  2.65s/it]

[Warning]: Max iteration during bisection


 12%|██████████████████                                                                                                                                        | 13/111 [00:33<04:05,  2.50s/it]

[Warning]: Max iteration during bisection
[Warning]: Max iteration during bisection
[Warning]: Max iteration during bisection
[Warning]: Max iteration during bisection


 14%|████████████████████▊                                                                                                                                     | 15/111 [00:45<06:25,  4.02s/it]

...

 51%|███████████████████████████████████████████████████████████████████████████████                                                                           | 57/111 [02:37<02:05,  2.32s/it]

[Warning]: Max iteration during bisection


 54%|███████████████████████████████████████████████████████████████████████████████████▏                                                                      | 60/111 [02:47<02:22,  2.80s/it]

[Warning]: Max iteration during bisection
[Warning]: Max iteration during bisection


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 111/111 [04:54<00:00,  2.65s/it]


In [59]:
dataloader_eval_adv = DataLoader(adv_datalist_eval, batch_size=128)

### 接下来尝试带入原来的框架

In [63]:
attacker = PGDAttack(surrogate=model)
dataloader_eval_adv = attacker.attack(eval_set, mask)

 12%|███████████████▎                                                                                                                    | 10/86 [00:23<03:00,  2.37s/it]

[Warning]: Max iteration during bisection


 50%|██████████████████████████████████████████████████████████████████                                                                  | 43/86 [01:47<01:39,  2.30s/it]

[Warning]: Max iteration during bisection


 78%|██████████████████████████████████████████████████████████████████████████████████████████████████████▊                             | 67/86 [02:47<00:41,  2.18s/it]

[Warning]: Max iteration during bisection
[Warning]: Max iteration during bisection


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 86/86 [03:40<00:00,  2.56s/it]


In [64]:
dataloader_eval_adv

In [65]:
# Accuracy on the adversarial data only
acc_adv_only, _ = eval_encoder(model, dataloader_eval_adv)

# Overall adversarial accuracy
acc_adv = acc_clean * acc_adv_only # T/all * Tadv/T = Tadv/all

print(f'(A): clean accuracy={acc_clean:.4f}, adversarial accuracy={acc_adv:.4f}')

(A): clean accuracy=0.7748, adversarial accuracy=0.7568


In [68]:
nnodes=3
torch.triu_indices(row=nnodes, col=nnodes, offset=1)

tensor([[0, 0, 1],
        [1, 2, 2]])